# multiple-output models
## such as a network that takes as input a series of social media posts from a single anonymous person and tries to predict attributes of that person, such as age, gender, and income level 

In [1]:
from keras import layers
from keras import Input
from keras.models import Model

In [2]:
vocabulary_size=50000
num_income_groups=10

post_input = Input(shape = (None,), dtype="int32", name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(post_input)

In [3]:
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)


In [4]:
age_prediction    = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation ='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation ='sigmoid', name='gender')(x)


In [5]:
model = Model(post_input,[age_prediction, income_prediction, gender_prediction])
model.compile(optimizer="rmsprop",
              loss = {'age':"mse",
                     "income":"categorical_crossentropy",
                     "gender":"binary_crossentropy"})

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 128)    32000128    embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, None, 128)    0           conv1d[0][0]                     
______________________________________________________________________________________________